In [1]:
import os
import csv
import json
import time
import pdfplumber
import pytesseract
import openai
from PIL import Image

# Mettez ici votre clé d'API OpenAI
openai.api_key =""
def pdf_has_text(pdf_path: str) -> bool:
    """
    Vérifie si un PDF contient une couche texte ou s'il s'agit d'un scan.
    """
    try:
        with pdfplumber.open(pdf_path) as pdf:
            if len(pdf.pages) == 0:
                return False
            text = pdf.pages[0].extract_text()
            return bool(text and len(text.strip()) > 10)
    except:
        return False

def extract_text_from_pdf(pdf_path: str) -> str:
    """
    Extrait le texte d'un PDF contenant une couche texte.
    """
    all_text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            txt = page.extract_text()
            if txt:
                all_text.append(txt)
    return "\n".join(all_text)

def ocr_pdf(pdf_path: str) -> str:
    """
    Effectue l'OCR sur un PDF scanné et retourne le texte extrait.
    """
    text_parts = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            image = page.to_image(resolution=300)
            pil_img = image.original
            page_text = pytesseract.image_to_string(pil_img, lang="fra")  # OCR en français
            if page_text.strip():
                text_parts.append(page_text)
    return "\n".join(text_parts)

def chunk_text(text, chunk_size=2000):
    """
    Découpe un texte en morceaux de taille maximale 'chunk_size'.
    """
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

def analyze_statuts_chunk(chunk_text: str, model="gpt-4"):
    """
    Analyse un chunk de texte avec OpenAI. Gère les erreurs de rate limit.
    """
    system_msg = (
        "Tu es un assistant qui analyse les statuts d'une entreprise. "
        "Tu dois extraire les informations suivantes et retourner un JSON strict :\n\n"
        "1) enterprise_name : nom de l'entreprise s'il est indiqué clairement dans les statuts.\n"
        "2) raison_d_etre : si elle est définie, sinon laisse vide.\n"
        "3) objectifs_sociaux : si mentionnés, sinon vide.\n"
        "4) objectifs_environnementaux : si mentionnés, sinon vide.\n"
        "5) mission_nature : choisis parmi 'RSE', 'transformative de l’entreprise', "
        "'transformative du secteur d’activité', ou 'Aucune'.\n\n"
        "Ne réponds pas avec du texte supplémentaire, uniquement un objet JSON valide, comme ceci :\n"
        "{\n"
        '  "enterprise_name": "...",\n'
        '  "raison_d_etre": "...",\n'
        '  "objectifs_sociaux": "...",\n'
        '  "objectifs_environnementaux": "...",\n'
        '  "mission_nature": "..." \n'
        "}\n"
    )

    user_msg = f"Voici le texte intégral (ou partiel) des statuts :\n\n{chunk_text}\n\nAnalyse et retourne le JSON demandé."

    attempt = 0
    while attempt < 5:  # Réessayer jusqu'à 5 fois en cas d'erreur de Rate Limit
        try:
            response = openai.ChatCompletion.create(
                model=model,
                temperature=0.1,
                messages=[
                    {"role": "system", "content": system_msg},
                    {"role": "user", "content": user_msg},
                ]
            )

            content = response.choices[0].message["content"].strip()
            if not content:
                print("⚠️ Réponse OpenAI vide !")
                return {}

            return json.loads(content)

        except openai.error.RateLimitError as e:
            wait_time = 10 * (attempt + 1)  # Attente progressive (10s, 20s, 30s...)
            print(f"⚠️ Rate Limit atteint, attente de {wait_time} secondes...")
            time.sleep(wait_time)
            attempt += 1

        except json.JSONDecodeError:
            print("⚠️ Erreur JSON : Réponse mal formatée")
            return {}

        except Exception as e:
            print("⚠️ Erreur OpenAI :", e)
            return {}

    print("❌ Échec après plusieurs tentatives")
    return {}

def analyze_statuts_openai(full_text: str, model="gpt-4"):
    """
    Gère le cas où 'full_text' dépasse la limite de tokens en segmentant (chunking) si nécessaire.
    """
    MAX_CHARS = 12000  # Seuil pour découper en chunks

    if len(full_text) > MAX_CHARS:
        print(f"Texte trop volumineux ({len(full_text)} caractères). Découpage en chunks...")
        text_chunks = chunk_text(full_text, chunk_size=2000)

        aggregated_result = {
            "enterprise_name": "",
            "raison_d_etre": "",
            "objectifs_sociaux": "",
            "objectifs_environnementaux": "",
            "mission_nature": ""
        }

        for idx, ch in enumerate(text_chunks):
            print(f"Analyse du chunk {idx+1}/{len(text_chunks)}...")
            partial_result = analyze_statuts_chunk(ch, model)

            for key in aggregated_result:
                if not aggregated_result[key] and partial_result.get(key):
                    aggregated_result[key] = partial_result[key]

        return aggregated_result
    else:
        return analyze_statuts_chunk(full_text, model)

def process_pdfs_in_folder(pdf_folder: str, output_csv: str, model="gpt-4"):
    """
    Parcourt chaque PDF d'un dossier, extrait (ou OCR si nécessaire) le texte,
    l'analyse via OpenAI et enregistre dans un CSV.
    """
    results = []
    pdf_files = [f for f in os.listdir(pdf_folder) if f.lower().endswith(".pdf")]

    for fname in pdf_files:
        pdf_path = os.path.join(pdf_folder, fname)
        print(f"Traitement du fichier : {pdf_path}")

        if pdf_has_text(pdf_path):
            text = extract_text_from_pdf(pdf_path)
        else:
            text = ocr_pdf(pdf_path)

        analysis = analyze_statuts_openai(text, model)

        row = {
            "fichier": fname,
            "enterprise_name": analysis.get("enterprise_name", ""),
            "raison_d_etre": analysis.get("raison_d_etre", ""),
            "objectifs_sociaux": analysis.get("objectifs_sociaux", ""),
            "objectifs_environnementaux": analysis.get("objectifs_environnementaux", ""),
            "mission_nature": analysis.get("mission_nature", "")
        }
        results.append(row)

    with open(output_csv, mode="w", encoding="utf-8", newline="") as f:
        fieldnames = [
            "fichier",
            "enterprise_name",
            "raison_d_etre",
            "objectifs_sociaux",
            "objectifs_environnementaux",
            "mission_nature"
        ]
        
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for r in results:
            writer.writerow(r)

    print(f"\n✅ Terminé ! Résultats enregistrés dans {output_csv}")
if __name__ == "__main__":
    
    DOSSIER_PDF = r"C:\Users\daffes\Desktop\projet_datathom"

    FICHIER_SORTIE = r"C:\Users\daffes\Desktop\projet_datathom\PDF\Resulta\missions_classees_openai.csv"
    
    # Utilisation de gpt-3.5-turbo pour économiser des tokens (optionnel)
    process_pdfs_in_folder(DOSSIER_PDF, FICHIER_SORTIE, model="gpt-3.5-turbo")

Traitement du fichier : C:\Users\daffes\Desktop\projet_datathom\22. Autorisation d'utilisation et d'exploitation de l'image d'un salarié du groupe AREP màj 2024.pdf
Traitement du fichier : C:\Users\daffes\Desktop\projet_datathom\8. Bulletin dispense d'affiliation de droit.pdf

✅ Terminé ! Résultats enregistrés dans C:\Users\daffes\Desktop\projet_datathom\PDF\Resulta\missions_classees_openai.csv
